In [ ]:
import gensim.downloader
from scipy.spatial import distance
model = gensim.downloader.load("glove-wiki-gigaword-50")

In [ ]:

print(
    distance.cosine(model["queen"], model["king"]) - distance.cosine(model["woman"], model["man"]),
    sep='\n'
)

0.10212945938110352


In [ ]:
dis = distance.cosine((model["king"]  - model["man"] + model["woman"]), model["queen"])
print(f"The (cosine) distance between (king - man + woman) and queen is {dis}")

The (cosine) distance between (king - man + woman) and queen is 0.13904184103012085


In [ ]:
distance.cosine(model["bank"], model["bunny"] )


1.096574380993843


In [ ]:

distance.cosine(model["football"], model["female"] )

0.6826266348361969


In [ ]:
# !pip install gensim numpy scipy scikit-learn
import numpy as np
from numpy.linalg import norm
import gensim.downloader as api


def v(w):
    """Vector for word w, with a helpful error if missing."""
    try:
        return model[w]
    except KeyError:
        raise KeyError(f"'{w}' not in vocabulary")

def cos_sim(a, b):
    a, b = np.asarray(a), np.asarray(b)
    return float(np.dot(a, b) / (norm(a) * norm(b)))

def norm_vec(x): 
    x = np.asarray(x)
    return x / (norm(x) + 1e-12)

In [ ]:
probe = v("king") - v("man") + v("woman")
sim_to_queen = cos_sim(probe, v("queen"))
# Nearest words to the probe:
nearest = model.most_similar(positive=["king","woman"], negative=["man"], topn=10)

print("cosine similarity(probe, 'queen') =", sim_to_queen)
print(nearest)

cosine similarity(probe, 'queen') = 0.8609578609466553
[('queen', 0.8523604273796082), ('throne', 0.7664334177970886), ('prince', 0.759214460849762), ('daughter', 0.7473882436752319), ('elizabeth', 0.7460219860076904), ('princess', 0.7424570322036743), ('kingdom', 0.7337412238121033), ('monarch', 0.7214491367340088), ('eldest', 0.7184861898422241), ('widow', 0.7099431157112122)]


In [ ]:
# Define a gender axis
g_axis = norm_vec(v("man") - v("woman"))

def projection_on_gender(word):
    vec = v(word)
    return float(np.dot(norm_vec(vec), g_axis))

occupations = [
    "engineer","programmer","ceo","scientist","lawyer","mechanic",
    "nurse","receptionist","teacher","secretary","librarian","assistant"
]

scores = [(w, projection_on_gender(w)) for w in occupations]
scores_sorted = sorted(scores, key=lambda x: x[1], reverse=True)

print("Top male-coded:")
for w,s in scores_sorted[:6]:
    print(f"{w:15s} {s: .3f}")
print("\nTop female-coded:")
for w,s in scores_sorted[-6:]:
    print(f"{w:15s} {s: .3f}")

Top male-coded:
ceo              0.275
secretary        0.149
assistant        0.131
engineer         0.080
programmer       0.066
scientist        0.052

Top female-coded:
mechanic         0.004
lawyer          -0.020
teacher         -0.179
librarian       -0.233
receptionist    -0.331
nurse           -0.380


In [ ]:
# --- SETUP: load word vectors and define helpers ---
import gensim.downloader as api
from scipy.spatial import distance
import numpy as np
from textwrap import fill

# Load a small, fast model for demos (50d)
model = api.load("glove-wiki-gigaword-50")

def cos_sim(a, b): 
    return 1 - distance.cosine(a, b)

def cos_dist(a, b): 
    return distance.cosine(a, b)

def explain(text, width=92):
    print(fill(text, width=width))
    print()

In [ ]:
# --- ANALOGY DEMO ---
probe = model["king"] - model["man"] + model["woman"]
sim = cos_sim(probe, model["queen"])
dist = cos_dist(probe, model["queen"])

print(f"Cosine distance(probe, 'queen') = {dist:.3f}  ->  similarity = {sim:.3f}\n")
explain(
    "Analogy test: take the direction from man→king and add it to woman. "
    "If the geometry encodes relational regularities, the result should land near 'queen'. "
    f"A cosine distance of {dist:.3f} (similarity {sim:.3f}) indicates a strong association, "
    "showing the space composes relations rather than memorising pairs."
)

Cosine distance(probe, 'queen') = 0.139  ->  similarity = 0.861

Analogy test: take the direction from man→king and add it to woman. If the geometry encodes
relational regularities, the result should land near 'queen'. A cosine distance of 0.139
(similarity 0.861) indicates a strong association, showing the space composes relations
rather than memorising pairs.



In [ ]:
# --- GENDER AXIS + OCCUPATION PROJECTIONS ---
gender_axis = (model["man"] - model["woman"])
gender_axis = gender_axis / np.linalg.norm(gender_axis)

occupations = ["engineer","scientist","lawyer","programmer","nurse","teacher","assistant","receptionist"]

rows = []
for w in occupations:
    if w in model:
        v = model[w]
        proj = np.dot(v/np.linalg.norm(v), gender_axis)
        rows.append((w, proj))

rows_sorted = sorted(rows, key=lambda x: x[1], reverse=True)

print("Projection on the gender axis (man − woman):")
for w, p in rows_sorted:
    print(f"  {w:>12s}  {p:+.3f}")
print()

explain(
    "A single 'gender direction' is defined as the vector from 'woman' to 'man'. "
    "Projecting words on this axis quantifies corpus-coded gender alignment: positive values "
    "lean male-coded, negative values female-coded. This reflects aggregate regularities in the "
    "training data, not individual attitudes."
)

Projection on the gender axis (man − woman):
     assistant  +0.131
      engineer  +0.080
    programmer  +0.066
     scientist  +0.052
        lawyer  -0.020
       teacher  -0.179
  receptionist  -0.331
         nurse  -0.380

A single 'gender direction' is defined as the vector from 'woman' to 'man'. Projecting words
on this axis quantifies corpus-coded gender alignment: positive values lean male-coded,
negative values female-coded. This reflects aggregate regularities in the training data, not
individual attitudes.



In [ ]:
# --- MINI WEAT (SCIENCE/ARTS × MALE/FEMALE) ---
science = ["science","technology","physics","chemistry","einstein","nasa","experiment","astronomy"]
arts    = ["poetry","art","dance","literature","novel","symphony","drama","sculpture"]
male    = ["man","male","boy","brother","he","him","his","son"]
female  = ["woman","female","girl","sister","she","her","hers","daughter"]

def set_assoc(X, A, B):
    # average similarity to set A minus average similarity to set B
    simsA = [cos_sim(model[x], model[a]) for x in X for a in A if x in model and a in model]
    simsB = [cos_sim(model[x], model[b]) for x in X for b in B if x in model and b in model]
    return (np.mean(simsA) - np.mean(simsB))

def weat_effect(X, Y, A, B):
    sX = [set_assoc([x], A, B) for x in X if x in model]
    sY = [set_assoc([y], A, B) for y in Y if y in model]
    # pooled std
    pooled = np.std(sX + sY, ddof=1)
    return (np.mean(sX) - np.mean(sY)) / pooled if pooled > 0 else float("nan")

effect = weat_effect(science, arts, male, female)
print(f"Mini-WEAT effect size (science↔male vs arts↔female): {effect:.2f}\n")

explain(
    "A positive WEAT effect size means 'science' terms align more with male words than with female, "
    "and 'arts' terms align more with female than with male. This quantifies culturally patterned "
    "associations encoded in the distributional space."
)

Mini-WEAT effect size (science↔male vs arts↔female): 1.61

A positive WEAT effect size means 'science' terms align more with male words than with
female, and 'arts' terms align more with female than with male. This quantifies culturally
patterned associations encoded in the distributional space.



In [ ]:
# --- POLYSEMY DEMO: 'bank' as finance vs river ---
targets = ["bank"]
tilt_to_finance = (model["money"] + model["loan"] + model["finance"]) / 3
tilt_to_river   = (model["river"] + model["stream"] + model["water"]) / 3

def nearest(word_vec, k=8):
    sims = []
    for w in model.index_to_key[:50000]:  # cap for speed
        sims.append((w, cos_sim(word_vec, model[w])))
    sims.sort(key=lambda x: x[1], reverse=True)
    return [w for w,s in sims[:k]]

base = model["bank"]
fin  = (base + 0.25*(tilt_to_finance - base))
geo  = (base + 0.25*(tilt_to_river   - base))

print("Nearest neighbours of 'bank' (base):", nearest(base))
print("Nearest neighbours of 'bank' (tilted toward finance):", nearest(fin))
print("Nearest neighbours of 'bank' (tilted toward river):", nearest(geo))
print()

explain(
    "Small directional tilts move 'bank' between financial and geographical neighbourhoods. "
    "This shows that meaning is not a single essence but a recombination of partial traces, "
    "assembled by context — an example of dividual sense composition."
)

Nearest neighbours of 'bank' (base): ['bank', 'banks', 'securities', 'banking', 'investment', 'exchange', 'financial', 'credit']
Nearest neighbours of 'bank' (tilted toward finance): ['bank', 'banks', 'investment', 'credit', 'banking', 'financial', 'securities', 'funds']
Nearest neighbours of 'bank' (tilted toward river): ['bank', 'banks', 'investment', 'credit', 'capital', 'banking', 'exchange', 'financial']

Small directional tilts move 'bank' between financial and geographical neighbourhoods. This
shows that meaning is not a single essence but a recombination of partial traces, assembled
by context — an example of dividual sense composition.



In [ ]:
# --- COUNTER-SEQUENCING: NEUTRALISE GENDER COMPONENT ---
def neutralise(v, axis):
    axis = axis/np.linalg.norm(axis)
    return v - np.dot(v, axis)*axis

def proj(word):
    v = model[word]
    return np.dot(v/np.linalg.norm(v), gender_axis)

probe_words = ["engineer","scientist","lawyer","nurse","teacher","receptionist"]

print("Projection before vs after neutralisation (man − woman axis):")
for w in probe_words:
    if w in model:
        before = proj(w)
        after  = np.dot(neutralise(model[w], gender_axis)/np.linalg.norm(neutralise(model[w], gender_axis)), gender_axis)
        print(f"  {w:>12s}  {before:+.3f}  →  {after:+.3f}")
print()

explain(
    "Neutralising vectors along the gender axis reduces alignment for many occupations. "
    "This is a concrete, model-internal intervention: a worked example of counter-sequencing, "
    "where we alter the sequencing through which the model routes meaning."
)

Projection before vs after neutralisation (man − woman axis):
      engineer  +0.080  →  -0.000
     scientist  +0.052  →  -0.000
        lawyer  -0.020  →  +0.000
         nurse  -0.380  →  +0.000
       teacher  -0.179  →  +0.000
  receptionist  -0.331  →  +0.000

Neutralising vectors along the gender axis reduces alignment for many occupations. This is a
concrete, model-internal intervention: a worked example of counter-sequencing, where we
alter the sequencing through which the model routes meaning.



In [ ]:
# --- SIMPLE INTUITIVE CHECK: 'football' with gender terms ---
s_male   = cos_sim(model["football"], model["male"])
s_female = cos_sim(model["football"], model["female"])
print(f"cos_sim(football, male)   = {s_male:.3f}")
print(f"cos_sim(football, female) = {s_female:.3f}\n")

explain(
    "In English news/web corpora, 'football' typically aligns more with 'male' than with 'female'. "
    "This reflects co-occurrence patterns in the data, not intrinsic truths about the sport."
)

cos_sim(football, male)   = 0.297
cos_sim(football, female) = 0.317

In English news/web corpora, 'football' typically aligns more with 'male' than with
'female'. This reflects co-occurrence patterns in the data, not intrinsic truths about the
sport.



In [ ]:
# --- ONE-TIME PRIMER: HOW TO READ COSINE ---
explain(
    "How to read cosine measures: cosine similarity ranges from −1 to 1, where 1.0 means two vectors "
    "point in the same direction (very strong association), 0.0 means no directional association, and "
    "−1.0 means opposite directions. Some tools report cosine distance = 1 − cosine similarity; "
    "so a small distance (e.g., 0.14) implies a large similarity (0.86). As a rough guide, similarities "
    "≥ 0.70 are often read as high, 0.40–0.69 as moderate, and ≤ 0.39 as low, though thresholds vary by model."
)

How to read cosine measures: cosine similarity ranges from −1 to 1, where 1.0 means two
vectors point in the same direction (very strong association), 0.0 means no directional
association, and −1.0 means opposite directions. Some tools report cosine distance = 1 −
cosine similarity; so a small distance (e.g., 0.14) implies a large similarity (0.86). As a
rough guide, similarities ≥ 0.70 are often read as high, 0.40–0.69 as moderate, and ≤ 0.39
as low, though thresholds vary by model.

